In [1]:
# Imports
import arcgis
from arcgis.gis import GIS
from getpass import getpass
import pandas as pd
import csv

https://developers.arcgis.com/python/sample-notebooks/inventory-organizational-content/

In [2]:
# Define source and target portals. Connect to them using accounts with administrative privileges:
source_password = getpass()
gis = GIS("y", username='x')
source_admin_username = 'x'

········
Enter password: ········


In [ ]:
# Validate
print("Successfully logged in as: " + gis.properties.user.username)

In [ ]:
org_users = gis.users.search()
print(f'{len(org_users)} users found')
org_users[:50]

In [ ]:
max_items=125
qe = f"type:Web Mapping Application, owner:{gis.users.me.username}"
content = gis.content.search(query=qe, outside_org=False, sort_field='title', max_items=max_items)

In [ ]:
content

In [ ]:
# return content for each user in org, compile into a single list
org_content = []

for user in org_users:
    qe = f"owner: {user.username}"
    user_content = gis.content.advanced_search(query=qe, max_items=-1)['results']
    org_content += user_content
    
print(f"{len(org_content)} items found in org")

In [ ]:
content_df = pd.DataFrame(org_content)
content_df.head()

In [ ]:
# use value_counts() to see how many items you have with a particular key:value pair
content_df.type.value_counts().head(10)

In [ ]:
view_columns = ['id','title','owner','type','access','url']
content_df[view_columns].head()

In [ ]:
# Creating masks with .apply and lambda functions
app_filter = content_df.type.apply(lambda x: x.endswith(('Application', 'StoryMap', 'Data Collection')))
apps_df = content_df[app_filter][view_columns]
apps_df

In [ ]:
# Creating masks with .apply and lambda functions
service_filter = content_df.type.apply(lambda x: x.endswith(('Service')))
service_df = content_df[service_filter][view_columns]
service_df

In [ ]:
# Exporting data to a csv
target_path = "Content_Applications.csv"
apps_df.to_csv(target_path)